# S2W6D2: 驾驭变长序列 (Advanced Batch Input)

**🎯 核心目标**: 解决深度学习中的“对齐”难题。
Tensor 必须是规整的矩形（矩阵），但人类语言的长短是不确定的。今天我们要掌握如何用 `Padding`, `Truncation` 和 `Attention Mask` 完美处理 Batch 数据，并理解 `offset_mapping` 在信息抽取任务中的关键作用。

## 1 理论：张量的“普鲁克拉斯提斯之床”

古希腊神话有个强盗叫普鲁克拉斯提斯，他把人放在床上，太长就截断，太短就拉长。这就是我们在 Batch Training 中做的事：

1.  **Padding (补齐)**: 无论句子多短，都补 0 到统一长度。
2.  **Truncation (截断)**: 无论句子多长，切掉超过 `max_length` 的部分。
3.  **Attention Mask (掩码)**: 这是最重要的补救措施。
      * **问题**: 补进去的 0 也是数字，如果不处理，它们会参与 `Softmax` 计算，导致概率分布出错。
      * **解法**: Mask 告诉模型：“这些位置是凑数的，把它们的注意力分数设为 $-\infty$。”

## 2 代码实战 (Hands-on)

首先，加载我们昨天保存到本地的 Tokenizer（**坚持使用本地资产**）。

In [1]:
import os
import torch
from transformers import BertTokenizer

# --- 加载本地模型资产 ---
# 此时我们已经不再依赖网络，也不用每次都写一堆下载逻辑
current_dir = os.getcwd()
model_path = os.path.abspath(os.path.join(current_dir, '../../data/pretrained_models/bert-base-chinese'))

tokenizer = BertTokenizer.from_pretrained(model_path)
print("本地Tokenizer加载完毕")

/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


本地Tokenizer加载完毕


### 2.1 批量编码的“全自动武器”

以前我们要手动 pad，现在 Hugging Face 提供了全自动工具。

In [2]:
# 模拟一个 Batch，长度差异巨大
batch_sentences = [
    "AI",                           # 极短
    "算法工程师需要掌握Transformer",    # 中等
    "在深度学习中，Batch Size 的大小会直接影响显存占用和梯度的稳定性，需要根据硬件仔细调整。" # 极长
]

# --- 核心调用 ---
encoded_input = tokenizer(
    batch_sentences,
    padding=True,           # 自动补齐到当前 Batch 中最长句子的长度 (Dynamic Padding)
    truncation=True,        # 超长截断
    max_length=20,          # 强制设定最大长度 (为了演示截断效果，设个小一点的值)
    return_tensors='pt'     # 返回 PyTorch Tensor
)

print("--- Input IDs (Shape) ---")
print(encoded_input['input_ids'].shape) 
# 观察: 应该是 [3, 20]。因为最长句被截断到 20，短句被 Pad 到 20。

print("\n--- Input IDs (Content) ---")
print(encoded_input['input_ids'])
# 观察: 短句后面全是 0 (Pad)，长句最后一位通常是 102 (SEP) 或者被强行截断

print("\n--- Attention Mask ---")
print(encoded_input['attention_mask'])
# 观察: 0 对应 Pad 的位置，1 对应真实内容

--- Input IDs (Shape) ---
torch.Size([3, 20])

--- Input IDs (Content) ---
tensor([[ 101,  100,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 5050, 3791, 2339, 4923, 2360, 7444, 6206, 2958, 2995,  100,  102,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 1762, 3918, 2428, 2110,  739,  704, 8024,  100,  100, 4638, 1920,
         2207,  833, 4684, 2970, 2512, 1510, 3227,  102]])

--- Attention Mask ---
tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


### 2.2 深度实验：Mask 真的有用吗？(Interview Killer)

这是一个非常经典的面试/Debug 场景：**证明 Attention Mask 的作用**。

我们来做一个实验：

1.  把一个短句 "Hello" 单独输入模型。
2.  把 "Hello" 加上一堆 Pad 输入模型。
3.  如果不看 Pad，这两次输入的**真实部分 (Hello)** 的输出特征应该是一模一样的。

*注：为了做这个实验，我们需要简单加载一下 Model（虽然这是明天的内容，但为了验证原理，我们先偷跑一下）。*


In [3]:
from transformers import BertModel

# 加载模型 (这里会临时下载一下模型权重，稍安勿躁，明天我们会本地化它)
# 如果你网络不好，这里可能会卡一下，可以先只看代码理解原理
model = BertModel.from_pretrained('bert-base-chinese')
model.eval() # 切换到评估模式 (关闭 Dropout)

# Case 1: 纯净的输入
input1 = tokenizer(["你好"], return_tensors='pt')
out1 = model(**input1).last_hidden_state

# Case 2: 带有大量 Padding 的输入
input2 = tokenizer(["你好"], padding='max_length', max_length=10, return_tensors='pt')
out2 = model(**input2).last_hidden_state

print("\n--- 实验结果对比 ---")
print(f"不带 Pad 的输出 Shape: {out1.shape}") # [1, 4, 768] (CLS, 你, 好, SEP)
print(f"带 Pad 的输出 Shape:   {out2.shape}") # [1, 10, 768]

# 核心验证: 比较前4个位置的数值差异
# 如果 Attention Mask 生效，这两个 Tensor 的前4行应该几乎完全一样
diff = (out1[0] - out2[0, :4]).abs().sum()
print(f"数值差异 (Diff): {diff.item()}")

# 结论: 
# 如果 diff 非常小 (e.g., < 1e-5)，说明 Mask 成功屏蔽了 Pad 的干扰。
# 如果没有 Mask，Pad 的 0 会参与 Softmax 计算，拉低 "你好" 的权重，导致特征发生变化。

'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 5db54e0c-8544-48e8-8c9a-b77d680719b2)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 21830b5f-9f63-4e24-8670-df052641374d)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
Retrying in 2s [Retry 2/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: ac33e644-5f27-4874-ab1a-3ab29054d2b9)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
Retrying in 4s [Retry 3/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: f8af05b3-135a-48a2-8d6f-371366b6fdb0)')' thrown while requesting HEAD ht

KeyboardInterrupt: 

## 3 进阶面试点：Offset Mapping (偏移映射)

这在 **信息抽取 (Information Extraction)**、**命名实体识别 (NER)**、**阅读理解 (QA)** 任务中是必考题。

  * **场景**: 简历解析。
      * **原文**: "Candidate: Huang Gaopeng, Master Degree."
      * **任务**: 提取 "Huang Gaopeng" 在原文中的起始和结束字符索引 (11, 24)。
  * **问题**: Tokenizer 把 "Huang Gaopeng" 拆成了 `['Hu', '##ang', 'Gao', '##peng']` (WordPiece)。Token 的索引和原文的字符索引**对不上**了。
  * **解法**: `return_offsets_mapping=True`。


In [4]:
text = "Apple公司的CEO是Tim Cook"
#
inputs = tokenizer(
    text,
    return_offsets_mapping=True,
    return_tensors='pt'
)

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
offsets = inputs['offset_mapping'][0]

print("\n--- Offset Mapping 实战 ---")
print(f"{'Token':<10} | {'Offset':<10} | {'Original Char'}")
print("-" * 35)

for token, offset in zip(tokens, offsets):
    start, end = offset.tolist()
    # [CLS] 和 [SEP] 的 offset 通常是 (0,0)
    origin_substr = text[start:end] if start != end else "[SPECIAL]"
    print(f"{token:<10} | {str((start, end)):<10} | {origin_substr}")

NotImplementedError: return_offset_mapping is not available when using Python tokenizers. To use this feature, change your tokenizer to one deriving from transformers.PreTrainedTokenizerFast. More information on available tokenizers at https://github.com/huggingface/transformers/pull/2674

## 4 ⚔️ 每日算法题 (LeetCode Daily)

今天的题目与 **Batch 处理** 和 **字符串操作** 高度相关。

### 🎯 题目: LeetCode 6. Z 字形变换 (Zigzag Conversion)

  * **难度**: Medium
  * **链接**: [LeetCode 6](https://leetcode.cn/problems/zigzag-conversion/)
  * **描述**: 将字符串按 Z 字形排列后重新读取。
  * **关联**: 这道题训练的是**索引映射 (Index Mapping)** 的能力。在处理 Transformer 的 **Positional Encoding** 或者设计特殊的 **Attention Mask**（如 Sparse Attention）时，这种对索引进行复杂变换的逻辑思维非常重要。

### ✅ 任务清单 (Action Items)

1.  **运行代码**: 跑通 2.1 的 Batch Padding 和 2.3 的 Offset Mapping。
2.  **理解实验**: 尤其是 Mask 那个实验，要理解为什么 Padding 不应该影响原有内容的计算结果。
3.  **LeetCode**: 完成 LeetCode 6。